In [1]:
#had a lot of issues trying to install the "imputeLCMD" package, so this was how I did it
# if (!requireNamespace("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")

# BiocManager::install(c("impute", "pcaMethods"))

# install.packages("imputeLCMD")

In [2]:
setwd('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1003. Ozone Sterols Healthy Asthma/P1003.3. Analyses/P1003.3.3. Ozone Responder Prediction/Input')
Output = ('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1003. Ozone Sterols Healthy Asthma/P1003.3. Analyses/P1003.3.2. Ozone Responder Prediction/Output')
cur_date = "030122"

library(readxl)
library(tidyverse)
library(gtsummary)
library(caret)
library(e1071)
library(Hmisc)
library(glmnet)
library(randomForest)
library(pROC)


#reading in files
ozone_data = data.frame(read_excel("Imputed_Ozone_Data_022822.xlsx")) 
demographics_data = data.frame(read_excel("SubjectInfo_011822.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: survival


Attaching package: ‘survival’


The following object is masked from ‘package:caret’:

    cluster


Loading required package: Formula


Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


Loading required package: Matrix


Attaching package: ‘Mat

In [3]:
head(ozone_data)
head(demographics_data)

,Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,Sample_Weight,817.00
2,4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,TCC,395000.00
3,4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,TCC_Weight,483.48
4,4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,Neutrophils_Percent,25.00
5,4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,Neutrophils,98750.00
6,4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,Neutrophils_Weight,121.00


,Original_Subject_No,Subject_No,Subject_ID,Sex,Race,Ethnicity,Age,BMI,Disease_Status,Visit_Season
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
1,4,1,AS_1,M,W,NH,30,23.3,AS,Fa
2,7,2,AS_2,F,W,NH,39,32.7,AS,Sp
3,11,3,AS_3,M,W,NH,23,23.0,AS,Fa
4,13,4,AS_4,M,W,NH,20,21.4,AS,Wi
5,15,5,AS_5,F,B,NH,20,34.5,AS,Su
6,16,6,AS_6,F,W,NH,25,25.2,AS,Su


In [4]:
plasma_baseline_ozone_data = full_join(demographics_data, ozone_data) %>%
    #only interested in plasma soluble mediators (cytokines/sterols) at baseline
    filter(Sample_Type == "P" & Time_Point == "Pre") %>%
    filter(Category == "Cytokine" | Category == "Sterol") %>%
    select(-c("Sample_Type", "Time_Point", "Category")) %>%
    distinct()
head(plasma_baseline_ozone_data)

Joining, by = c("Original_Subject_No", "Subject_No", "Subject_ID", "Disease_Status")



,Original_Subject_No,Subject_No,Subject_ID,Sex,Race,Ethnicity,Age,BMI,Disease_Status,Visit_Season,Inflammatory_Responder,Lung_Responder,Variable,Value
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,4,1,AS_1,M,W,NH,30,23.3,AS,Fa,NR,NR,IFNg,8.0557406
2,4,1,AS_1,M,W,NH,30,23.3,AS,Fa,NR,NR,IL10,0.3476302
3,4,1,AS_1,M,W,NH,30,23.3,AS,Fa,NR,NR,IL12p70,0.4572941
4,4,1,AS_1,M,W,NH,30,23.3,AS,Fa,NR,NR,IL13,7.7955008
5,4,1,AS_1,M,W,NH,30,23.3,AS,Fa,NR,NR,IL1b,0.2763808
6,4,1,AS_1,M,W,NH,30,23.3,AS,Fa,NR,NR,IL2,0.5728830


Seeing if there are statistical differences in predictors stratified by inflammatory or lung status.

In [5]:
#first pivoting wider to make the tbl_summary function work
wider_data = pivot_wider(plasma_baseline_ozone_data,
                         #not actually taking the mean of multiple values
                         #did this bc cols would be type list rather than double
                         names_from = Variable, values_from = Value, values_fn = mean)

head(wider_data)

Original_Subject_No,Subject_No,Subject_ID,Sex,Race,Ethnicity,Age,BMI,Disease_Status,Visit_Season,⋯,SecoB,bEP_Chol,Lan,Zym,DHL,Des,dHLan,Zyme,Lath,Chol
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4,1,AS_1,M,W,NH,30,23.3,AS,Fa,⋯,1.816,25.669,0.171,0.057,0.224,0.641,0.004,3.014,1.969,1110.442
7,2,AS_2,F,W,NH,39,32.7,AS,Sp,⋯,112.521,17.867,0.062,0.046,0.106,0.286,0.015,2.508,1.287,673.328
11,3,AS_3,M,W,NH,23,23.0,AS,Fa,⋯,6.686,8.066,0.052,0.046,0.094,0.279,0.005,1.549,1.200,537.509
13,4,AS_4,M,W,NH,20,21.4,AS,Wi,⋯,48.416,10.132,0.013,0.046,0.085,0.297,0.016,1.833,1.062,482.396
15,5,AS_5,F,B,NH,20,34.5,AS,Su,⋯,13.006,13.090,0.117,0.060,0.116,0.255,0.020,3.628,1.795,586.750
16,6,AS_6,F,W,NH,25,25.2,AS,Su,⋯,36.458,14.331,0.022,0.034,0.077,0.272,0.006,1.914,0.966,639.636


These p values are relatively high, so I'm not expecting the model to predict well. 

In [6]:
#summary statistics df
wider_data %>%
  tbl_summary(by = Inflammatory_Responder, missing = "no", 
  include = colnames(wider_data[13:53]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% #adding p value from anova
  as_tibble()

**Characteristic**,**N**,"**NR**, N = 8","**R**, N = 16",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
IFNg,24,9.7 (8.9),5.3 (3.1),0.085
IL10,24,0.52 (0.48),0.38 (0.14),0.3
IL12p70,24,0.31 (0.10),0.32 (0.23),0.9
IL13,24,5.97 (1.00),6.84 (3.77),0.5
IL1b,24,NA,NA,0.6
0.14162,NA,7 (88%),15 (94%),NA
0.276380750999999,NA,1 (12%),0 (0%),NA
0.425723693999999,NA,0 (0%),1 (6.2%),NA
IL2,24,0.17 (0.19),0.19 (0.22),0.8


In [7]:
wider_data %>%
  tbl_summary(by = Lung_Responder, missing = "no", 
  include = colnames(wider_data[13:53]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% #adding p value from anova
  as_tibble()

**Characteristic**,**N**,"**NR**, N = 11","**R**, N = 13",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
IFNg,24,6.8 (5.9),6.8 (6.1),>0.9
IL10,24,0.34 (0.09),0.50 (0.39),0.2
IL12p70,24,0.30 (0.13),0.33 (0.24),0.7
IL13,24,5.74 (1.60),7.23 (3.93),0.3
IL1b,24,NA,NA,0.7
0.14162,NA,10 (91%),12 (92%),NA
0.276380750999999,NA,1 (9.1%),0 (0%),NA
0.425723693999999,NA,0 (0%),1 (7.7%),NA
IL2,24,0.14 (0.17),0.22 (0.24),0.4


# Which demographic information and/or soluble mediators in plasma collected at baseline best predict ozone-responder status in asthmatics and non-asthmatics?

This question will be tackled using 4 supervised machine learning methods assessing both regression and classification...

1. K Nearest Neighbor (KNN)
2. Random Forest
3. Support Vector Machine (SVM)
4. Penalized Logistic Regression

In [6]:
#making responder columns binary denoted by 0 = non-responder and 1 = responder and covariates
wider_data = wider_data %>%
    mutate(Inflammatory_Responder = relevel(factor(ifelse(Inflammatory_Responder == "R", 1, 0)), ref = "0")) %>%
    mutate(Lung_Responder = relevel(factor(ifelse(Lung_Responder == "R", 1, 0)), ref = "0")) %>%
    #for some reason RF didn't this variable so I changed it
    rename(X25OH_VitD = `25OH_VitD`) %>%
    mutate(Sex = relevel(factor(ifelse(Sex == "M", 1, 0)), ref = "0")) %>%
    mutate(Race = relevel(factor(ifelse(Sex == "M", 1,
                                        ifelse(Sex == "As", 2,
                                        ifelse(Sex == "B", 3, 0)))), ref = "0")) %>%
    mutate(Ethnicity = relevel(factor(ifelse(Ethnicity == "H", 1, 0)), ref = "0")) %>%
    mutate(Disease_Status = relevel(factor(ifelse(Disease_Status == "AS", 1, 0)), ref = "0")) %>%
    mutate(Visit_Season = relevel(factor(ifelse(Visit_Season == "Su", 3, 
                                               ifelse(Visit_Season == "Sp", 2,
                                               ifelse(Visit_Season == "Fa", 1, 0)))), ref = "0"))

head(wider_data)

Original_Subject_No,Subject_No,Subject_ID,Sex,Race,Ethnicity,Age,BMI,Disease_Status,Visit_Season,⋯,SecoB,bEP_Chol,Lan,Zym,DHL,Des,dHLan,Zyme,Lath,Chol
<dbl>,<dbl>,<chr>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4,1,AS_1,1,0,0,30,23.3,1,1,⋯,1.816,25.669,0.171,0.057,0.224,0.641,0.004,3.014,1.969,1110.442
7,2,AS_2,0,0,0,39,32.7,1,2,⋯,112.521,17.867,0.062,0.046,0.106,0.286,0.015,2.508,1.287,673.328
11,3,AS_3,1,0,0,23,23.0,1,1,⋯,6.686,8.066,0.052,0.046,0.094,0.279,0.005,1.549,1.200,537.509
13,4,AS_4,1,0,0,20,21.4,1,0,⋯,48.416,10.132,0.013,0.046,0.085,0.297,0.016,1.833,1.062,482.396
15,5,AS_5,0,0,0,20,34.5,1,3,⋯,13.006,13.090,0.117,0.060,0.116,0.255,0.020,3.628,1.795,586.750
16,6,AS_6,0,0,0,25,25.2,1,3,⋯,36.458,14.331,0.022,0.034,0.077,0.272,0.006,1.914,0.966,639.636


In [7]:
inflamm_resp_df = wider_data[,11:53] %>% select(-Lung_Responder)
inflamm_resp_covariates_df = wider_data[,4:53] %>% select(-Lung_Responder)
lung_resp_df = wider_data[,12:53]
lung_resp_covariates_df = wider_data[,4:53] %>% select(-Inflammatory_Responder)

head(lung_resp_covariates_df)

Sex,Race,Ethnicity,Age,BMI,Disease_Status,Visit_Season,Lung_Responder,IFNg,IL10,⋯,SecoB,bEP_Chol,Lan,Zym,DHL,Des,dHLan,Zyme,Lath,Chol
<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,0,30,23.3,1,1,0,8.055741,0.3476302,⋯,1.816,25.669,0.171,0.057,0.224,0.641,0.004,3.014,1.969,1110.442
0,0,0,39,32.7,1,2,0,5.456212,0.1802846,⋯,112.521,17.867,0.062,0.046,0.106,0.286,0.015,2.508,1.287,673.328
1,0,0,23,23.0,1,1,1,2.902175,0.4177936,⋯,6.686,8.066,0.052,0.046,0.094,0.279,0.005,1.549,1.200,537.509
1,0,0,20,21.4,1,0,0,3.860980,0.3901140,⋯,48.416,10.132,0.013,0.046,0.085,0.297,0.016,1.833,1.062,482.396
0,0,0,20,34.5,1,3,0,2.652720,0.2658850,⋯,13.006,13.090,0.117,0.060,0.116,0.255,0.020,3.628,1.795,586.750
0,0,0,25,25.2,1,3,0,22.007931,0.4377489,⋯,36.458,14.331,0.022,0.034,0.077,0.272,0.006,1.914,0.966,639.636


# Random Forest
- an ensemble learning method operating by constructing a multitude of decision trees at training time, which uses multiple methods to obtain a better predictive performance and includes bagging and random forest
- algorithm uses a bootstrop aggregation, to reduce overfitting the training datset but only a subset of the features are used hence decorrelation of predictors

In [8]:
rf_classification = function(dataset, outcome, pred_outcome){
    #setting for reproducibility
    set.seed(12)
    #splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV

    metrics = data.frame()
    variable_importance_df = data.frame()
    
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        ntree_values = c(50, 250, 500) #number of trees (not sure if we wanted something different)
        p = dim(dataset)[2] - 1 #number of variables in dataset
        mtry_values = c(p/2, sqrt(p), p)

        #will use ntree and mtry values to determine which combination yields the smallest MSE
        reg_rf_pred_tune = list()
        rf_OOB_errors = list()
        rf_error_df = data.frame()
        for (j in 1:length(ntree_values)){
            for (k in 1:length(mtry_values)){
                reg_rf_pred_tune[[k]] = randomForest(as.formula(paste0(outcome, "~.")), data = data_train, 
                                                     ntree = ntree_values[j], mtry = mtry_values[k])
                rf_OOB_errors[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                       "OOB_errors" = reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
                rf_error_df = rbind(rf_error_df, rf_OOB_errors[[k]])
            }
        }

        #finding the lowest OOB error using best number of predictors at split and refitting OG tree
        best_oob_errors <- which(rf_error_df$OOB_errors == min(rf_error_df$OOB_errors))

        reg_rf <- randomForest(as.formula(paste0(outcome, "~.")), data = data_train,
        #need to go back and figure out how the number of trees/ variables make a difference even though
        #the errors are the same!!!!
                               ntree = rf_error_df$Tree.Number[min(best_oob_errors)], 
                               mtry = rf_error_df$Variable.Number[min(best_oob_errors)])

        #predicting on test set
        data_test[[pred_outcome]] = predict(reg_rf, newdata = data_test, type = "response")


        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

        #calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        #extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
        
        #extracting variable importance
        var_importance_values = data.frame(importance(reg_rf)) %>%
            rownames_to_column(var = "MF")
        variable_importance_df = rbind(variable_importance_df, var_importance_values)
   
        #adding values to df
        metrics = rbind(metrics, matrix_values)
    }
    
    #taking averages/sd 
    metrics = metrics %>%
        summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
    
    variable_importance_df = variable_importance_df %>%
        group_by(MF) %>%
        summarise(MeanDecreaseGini = mean(MeanDecreaseGini)) %>%
        #sorting by most important variables
        arrange(-MeanDecreaseGini)
  
    #return training set, matrix and variable importance values
    return(list(data_train, metrics, variable_importance_df))

}

In [9]:
#calling fn
rf_values_inflamm_resp = rf_classification(inflamm_resp_df, "Inflammatory_Responder", "pred_Inflammatory_Responder")
rf_values_lung_resp = rf_classification(lung_resp_df, "Lung_Responder", "pred_Lung_Responder")
rf_values_inflamm_resp_covariates = rf_classification(inflamm_resp_covariates_df, "Inflammatory_Responder", "pred_Inflammatory_Responder")
rf_values_lung_resp_covariates = rf_classification(lung_resp_covariates_df, "Lung_Responder", "pred_Lung_Responder")

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [17]:
#viewing results
rf_confusion_matrix = data.frame(Model = c(rep(c("RF Classification"), times = 4)), 
                                 Response = c(rep(c("Inflammatory"), times = 2), rep(c("Lung"), times = 2)),
                                 Covariates = c(rep(c("No", "Yes"), times = 2)), Kernel = c(rep(c("NA"), times = 4)), rbind(rf_values_inflamm_resp[[2]], 
                                rf_values_inflamm_resp_covariates[[2]], rf_values_lung_resp[[2]], rf_values_lung_resp_covariates[[2]]))
rf_confusion_matrix

#viewing most significant features
head(rf_values_inflamm_resp[[3]])
head(rf_values_inflamm_resp_covariates[[3]])
head(rf_values_lung_resp[[3]]) 
head(rf_values_lung_resp_covariates[[3]]) 

Model,Response,Covariates,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF Classification,Inflammatory,No,NA,0.4666667,0.9333333,0.0000000,0.6566667,0.4666667
RF Classification,Inflammatory,Yes,NA,0.5166667,0.9333333,0.1000000,0.6866667,0.5166667
RF Classification,Lung,No,NA,0.7333333,0.8333333,0.6333333,0.7333333,0.7333333
RF Classification,Lung,Yes,NA,0.6000000,0.6666667,0.5333333,0.6166667,0.6000000


MF,MeanDecreaseGini
<chr>,<dbl>
IP10,0.8793780
IL8,0.7826965
IL5,0.6637214
MPO,0.5260840
TNFb,0.4634499
Chol,0.4183570


MF,MeanDecreaseGini
<chr>,<dbl>
IP10,0.5899506
IL8,0.5838754
IL5,0.5743383
IFNg,0.4407599
Chol,0.4106531
MPO,0.3989589


MF,MeanDecreaseGini
<chr>,<dbl>
Chol,1.4365670
X25OH_VitD,0.8685508
dHLan,0.8585950
IL15,0.7775042
Zym,0.6590335
bEP_Chol,0.4080964


MF,MeanDecreaseGini
<chr>,<dbl>
Chol,1.0736041
IL15,0.7949095
Zym,0.7351838
dHLan,0.6833614
X25OH_VitD,0.4843868
DHL,0.4546672


# Support Vector Machine (SVM)
- supervised learning models that can predict continuous (regression) or grouped/dichotomous (classification) outcomes
- predicts by projecting predictors onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)
- does a better job at handling a large number of predictors since p > n

In [12]:
#model errors are calculated to assess the predictive accuracy of the model
svm_classification = function(dataset, model, outcome, pred_outcome, gamma_values, cost_values, elsilon_values){
  
    #setting seed for reproducibility
    set.seed(12)
    
    #splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV
    metrics = data.frame()
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        #now pruning parameters (based on the training dataset to prevent overfitting)
        svr_tune <- tune(svm, as.formula(paste0(outcome, "~.")), data = data_train, kernel = model, 
                       ranges = list(elsilon = elsilon_values, cost = cost_values, gamma = gamma_values)) 

        #choosing best model
        best_svm <- svr_tune$best.model

        #predicting with tuned parameters 
        data_test[[pred_outcome]] <- predict(best_svm, newdata = data_test, type = "response")

        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                 positive = "1")
 
        #calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        #extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
   
        #adding values to df
        metrics = rbind(metrics, matrix_values)
  }
  
  #taking averages/sd 
  metrics = metrics %>%
    summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
  
  return(metrics)
}

In [13]:
#calling fn
#linear kernel
svm_linear_inflamm = svm_classification(inflamm_resp_df, "linear", "Inflammatory_Responder", "pred_Inflammatory_Responder",
                                             0.035, 1:5, seq(0,1,0.2))
svm_linear_lung = svm_classification(lung_resp_df, "linear", "Lung_Responder", "pred_Lung_Responder",
                                             0.035, 1:5, seq(0,1,0.2))
#radial
svm_radial_inflamm = svm_classification(inflamm_resp_df, "radial", "Inflammatory_Responder", "pred_Inflammatory_Responder",
                                             0.035, 1:5, seq(0,1,0.2))
svm_radial_lung = svm_classification(lung_resp_df, "radial", "Lung_Responder", "pred_Lung_Responder",
                                             0.035, 1:5, seq(0,1,0.2))
#polynomial
#elison value is fixed at 0.1, but I left it so I didn't have to create a new function
svm_polynomial_inflamm = svm_classification(inflamm_resp_df, "polynomial", "Inflammatory_Responder", 
                                             "pred_Inflammatory_Responder", 0.035, 1:5, 0.1)
svm_polynomial_lung = svm_classification(lung_resp_df, "polynomial", "Lung_Responder", "pred_Lung_Responder",
                                             0.035, 1:5, 0.1)

Setting levels: control = 0, case = 1

Setting direction: controls > cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.ac

Setting direction: controls < cases

Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = s

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘IL1b’ constant. Cannot scale data.”
Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Se

In [16]:
#creating one combined df
svm_final_df = data.frame(rbind(svm_linear_inflamm, svm_radial_inflamm, svm_polynomial_inflamm,
                           svm_linear_lung, svm_radial_lung, svm_polynomial_lung))
svm_final_df = data.frame(Model = c(rep(c("SVM Classification"), times = 6)),
                          Dataset = c(rep(c("Inflammatory"), times = 3), rep(c("Lung"), times = 3)),
                      Kernel = rep(c("linear", "radial", "polynomial"), times = 2), svm_final_df)
svm_final_df

Model,Dataset,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Original SVM,Inflammatory,linear,0.4083333,0.6166667,0.2000000,0.5366667,0.6083333
Original SVM,Inflammatory,radial,0.5000000,1.0000000,0.0000000,0.6733333,0.5000000
Original SVM,Inflammatory,polynomial,0.5333333,0.8666667,0.2000000,0.7033333,0.5333333
Original SVM,Lung,linear,0.7166667,0.6000000,0.8333333,0.8833333,0.7166667
Original SVM,Lung,radial,0.6333333,0.6333333,0.6333333,NaN,0.6333333
Original SVM,Lung,polynomial,0.4833333,0.7666667,0.2000000,0.5333333,0.4833333


# K Nearest Neighbor

- seeks to choose the model that minimizes MSE or the difference with the true mean. The model is flexible, which will make it more likely to overfit the training dataset and be less generalizable to other datasets.
- works best for continuous predictors since it groups based on Euclidean distance (a different approach is needed for continuous and dichotomous predictors)

In [36]:
#model errors are calculated to assess the predictive accuracy of the model
knn_classification = function(dataset, outcome, pred_outcome){

    #setting seed for reproducibility
    set.seed(12)

    #splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV
    knn_errors = data.frame()

    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        #training algorithm 
        knn_model = train(as.formula(paste0(outcome, "~.")), data = data_train, method = 'knn', 
                          #trControl = trainControl(method = "cv"), 
                          preProcess = c("center", "scale"))
        #print(knn_model)

        #plotting 
        #print(plot(knn_model))

        #assessing prediction
        data_test[[pred_outcome]] = predict(knn_model, newdata = data_test)
        matrix = confusionMatrix(data_test[[pred_outcome]], data_test[[outcome]])
        
        #calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        #extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)


        #adding values to df
        knn_errors = rbind(matrix_values, knn_errors)
    }

    
    #taking averages/sd 
    knn_errors = knn_errors %>%
        summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))

   return(knn_errors)
}

In [37]:
#calling fn
knn_values_inflamm_resp = knn_classification(inflamm_resp_df, "Inflammatory_Responder", "pred_Inflammatory_Responder")
knn_values_lung_resp = knn_classification(lung_resp_df, "Lung_Responder", "pred_Lung_Responder")

Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0

Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0

Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: IL1b”
Warning message in preProcess.default(thresh = 0

In [38]:
#viewing results
knn_confusion_matrix = data.frame(Model = c(rep(c("KNN Classification"), times = 2)), Response = c(rep(c("Inflammatory", "Lung"), times = 1)),
                                 Kernel = c(rep(c("NA"), times = 2)),rbind(knn_values_inflamm_resp, knn_values_lung_resp))
knn_confusion_matrix

Model,Response,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
KNN,Inflammatory,NA,0.5000000,0.0000000,1.0000000,NaN,0.5000000
KNN,Lung,NA,0.5833333,0.4333333,0.7333333,NaN,0.5833333


# Penalized Logistic Regression

- removes highly correlated variables, before performing logistic regression

In [25]:
#works by removing highly correlated variables prior to running logistric regression
cv_lm_lasso = cv.glmnet(x = data.matrix(inflamm_resp_df[,2:42]), y = unlist(inflamm_resp_df$Inflammatory_Responder), 
                        alpha = 1, family = "binomial")#1 denotes lasso

#fitting model on the whole data set
lm_lasso = glmnet(x = data.matrix(inflamm_resp_df[,2:42]),y = unlist(inflamm_resp_df$Inflammatory_Responder), 
                  alpha = 1, lambda = cv_lm_lasso$lambda.min, family = "binomial")

#regression estimates
coef(lm_lasso)

Warning message in lognet(xd, is.sparse, ix, jx, y, weights, offset, alpha, nobs, :
“one multinomial or binomial class has fewer than 8  observations; dangerous ground”
Warning message in lognet(xd, is.sparse, ix, jx, y, weights, offset, alpha, nobs, :
“one multinomial or binomial class has fewer than 8  observations; dangerous ground”
Warning message in lognet(xd, is.sparse, ix, jx, y, weights, offset, alpha, nobs, :
“one multinomial or binomial class has fewer than 8  observations; dangerous ground”
Warning message in lognet(xd, is.sparse, ix, jx, y, weights, offset, alpha, nobs, :
“one multinomial or binomial class has fewer than 8  observations; dangerous ground”
Warning message in lognet(xd, is.sparse, ix, jx, y, weights, offset, alpha, nobs, :
“one multinomial or binomial class has fewer than 8  observations; dangerous ground”
Warning message in lognet(xd, is.sparse, ix, jx, y, weights, offset, alpha, nobs, :
“one multinomial or binomial class has fewer than 8  observations; dang

42 x 1 sparse Matrix of class "dgCMatrix"
                       s0
(Intercept)  6.931472e-01
IFNg         .           
IL10         .           
IL12p70      .           
IL13         .           
IL1b         .           
IL2          .           
IL4          .           
IL6          .           
IL8          .           
TNFa         .           
GMCSF        .           
IL12p40      .           
IL15         .           
IL16         .           
IL17         .           
IL1a         .           
IL5          .           
IL7          .           
TNFb         .           
VEGF         .           
Eotaxin      .           
Eotaxin3     .           
IP10         .           
MCP1         .           
MCP4         .           
MDC          .           
MIP1a        .           
MIP1b        .           
TARC         .           
MPO          .           
X25OH_VitD   .           
SecoB        .           
bEP_Chol     .           
Lan          .           
Zym          .        

This ended up not being useful due to the small sample size. 

In [ ]:
#combining into 1 df
final_df = rbind(rf_confusion_matrix, svm_final_df, knn_confusion_matrix)
final_df